In [1]:
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier 
from sklearn.ensemble import BaggingClassifier
from sklearn.mixture import BayesianGaussianMixture
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.mixture import GaussianMixture 
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier 
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV 
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import NuSVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier 
from sklearn.svm import SVC 
from sklearn.model_selection import cross_val_score 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score 
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid 
from sklearn.metrics import classification_report 
from sklearn import preprocessing 
import category_encoders as ce 
from sklearn.externals import joblib 
estimators = [x for x in range(50, 110, 10)] 
fewer_estimators = [x for x in range(10, 60, 5)]  
estimators_1_to_20 = [x for x in range(1, 20, 2)]   
learning_rate = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

## Reading in Data:
#### Bills: Bill metadata and computed subject areas
#### Bill ideologies: Bill ideology scores computed from subject-specific scores of cosponsors 
#### Member ideologies: Subject-specific ideology scores for all legislators as computed from their vote records in each subject 

In [2]:
bills = pd.read_csv('data_store/dataframes/full_congress_level_df8.csv')
bill_ides = pd.read_csv('data_store/ides_by_bill/ide_distance_df.csv') 
member_ides = pd.read_csv('all_member_ideologies.csv') 

/home/william/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (2,3,4,7,8,9,12,16,17,18,20,21,24,25,26,29,37,38,39,42,45,46,47,50,53,54,55,58,61,62,63,66,69,70,71,74,77,78,79,82,85,86,87,90,93,94,95,98,101,102,103,106,109,110,111,114,115,116,117,144,145,146,150,151,152,154,155,158,159,160,163,166,167,168,170,171,174,175,176,178,179,182,183,184,187,194,195,196,200,201,202,206,208,211,212,213,215,216,221,222,223,226,229,230,231,234) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
member_id_list = list(member_ides['bioguide_id'].unique())

### Function to assemble each member's member-specific dataframe for the purpose of applying their member-specific vote prediction model

In [4]:
def vote_df_builder(bioguide_id, bill_df, bill_ides_df):
    member_df = pd.read_csv('data_store/member_ideology/' +str(
        bioguide_id) +'_ideology_df.csv')
    member_df['congress'] = member_df['congress'].astype(int)
    congress_numbers = member_df['congress'].unique()
    vote_dfs = []
    for congress_number in congress_numbers:
        sub_df = pd.read_csv('data_store/votes/congress_'+str(
            congress_number)+ '_member_votes.csv')
        vote_dfs.append(sub_df)
    vote_df = pd.concat(vote_dfs)
    vote_df = vote_df[vote_df['bioguide_id'] == bioguide_id].reset_index(drop=True)
    results_list = []
    for string in vote_df['bill_id']:
        if 'E' in string:
            results_list.append(1)
        elif 'P' in string:
            results_list.append(1)
        elif 'J' in string:
            results_list.append(1)
        else:
            results_list.append(0) 
    vote_df['check_col'] = results_list
    vote_df = vote_df[vote_df['check_col'] == 0]
    vote_df = vote_df.drop('check_col', axis=1)
    vote_df = vote_df.reset_index(drop=True)
    vote_df = vote_df[['vote_id', 'vote']] 
    merged0 = pd.merge(member_df, vote_df, on='vote_id', how='left')
    bill_df = bill_df[['bill_id', 'major_label 1', 'major_label 2', 'major_label 3']]
    merged = pd.merge(merged0, bill_df, on='bill_id', how='left')
    bill_ids = []
    vote_ids = []
    member_label_1_scores = []
    member_label_2_scores = []
    member_label_3_scores = []
    merged['major_label 1'] = merged['major_label 1'].astype(int)
    merged['major_label 2'] = merged['major_label 2'].astype(int)
    merged['major_label 3'] = merged['major_label 3'].astype(int)
    merged.dropna(axis=0, inplace=True)
    merged.reset_index(drop=True, inplace=True)
    for row in range(len(merged)):
        label_1 = 'topic_' + str(merged['major_label 1'][row].item()) 
        label_2 = 'topic_' + str(merged['major_label 2'][row].item())
        label_3 = 'topic_' + str(merged['major_label 3'][row].item())
        vote_ids.append(merged['vote_id'][row])
        bill_ids.append(merged['bill_id'][row])
        member_label_1_scores.append(merged[label_1][row])
        member_label_2_scores.append(merged[label_2][row])
        member_label_3_scores.append(merged[label_3][row]) 
    output_dict = {'vote_id': vote_ids, 'bill_id': bill_ids,
                   'member label 1 score': member_label_1_scores,
                   'member label 2 score': member_label_2_scores,
                   'member label 3 score': member_label_3_scores}
    label_df = pd.DataFrame(output_dict)
    output_df0 = pd.merge(merged, label_df, on=['vote_id', 'bill_id'], how='left')
    pre_merge_bill_ides = bill_ides_df.drop('congress', axis=1) 
    output_df = pd.merge(output_df0, pre_merge_bill_ides, on='bill_id', how='left')
    x1 = 'bill avg_label_1 ideology'
    y1 = 'bill avg_label_2 ideology'
    z1 = 'bill avg_label_3 ideology'
    x2 = 'member label 1 score' 
    y2 = 'member label 2 score'
    z2 = 'member label 3 score'
    drop_cols = ['thomas_id', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5',
                'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10', 'topic_12',
                'topic_13', 'topic_14', 'topic_15', 'topic_16', 'topic_17', 'topic_18',
                'topic_19', 'topic_20', 'topic_21', 'topic_99', 'member label 1 score',
                'member label 2 score', 'member label 3 score', 'bill avg_label_1 ideology',
                'bill avg_label_2 ideology', 'bill avg_label_3 ideology']
    
    # Calculating the euclidean distance from legislator to bill 
    output_df['ideological distance'] = np.sqrt((output_df[x1]-output_df[x2])**2 +(
    output_df[y1]-output_df[y2])**2 + (output_df[z1]-output_df[z2])**2)
    output_df = output_df.drop(drop_cols, axis=1)
    output_df.dropna(axis=0, inplace=True)
    output_df['major_label 1'] = output_df['major_label 1'].astype(str)
    output_df['major_label 2'] = output_df['major_label 2'].astype(str)
    output_df['major_label 3'] = output_df['major_label 3'].astype(str)
    return output_df 

In [5]:
def onehot_encoder(df):
    y = df['vote']
    X = df.drop('vote', axis=1) 
    ohe = ce.OneHotEncoder(handle_unknown='ignore', use_cat_names=True)
    df_ohe = ohe.fit_transform(X)
    df_ohe['vote'] = y 
    return df_ohe  

### Model and hyperparameter dictionary used in gridsearch to identify and optimize the best-scoring model for each unique legislator 

In [45]:
model_dict = {1:
              {'name': 'AdaBoostClassifier', 
               'model': AdaBoostClassifier(),
               'params': {'n_estimators': [np.arange(50, 110, 10)],
                          'learning_rate': [learning_rate]}},
              2:
              {'name': 'XGBClassifier',
               'model': XGBClassifier(),
               'params': {'n_estimators': [np.arange(50, 110, 10)],
                          'learning_rate': [learning_rate],
                          'booster': [['gbtree'], ['gblinear'], ['dart']]}},
              3:
              {'name': 'BaggingClassifier',
               'model': BaggingClassifier(),
               'params': {'n_estimators': [np.arange(10, 105, 5)]}},
              
              4:
              {'name': 'BernoulliNB',
               'model': BernoulliNB(),
               'params': {'alpha': [[1.0], [0]]}},
              
              5:
              {'name': 'DecisionTreeClassifier',
               'model': DecisionTreeClassifier(),
               'params': {'criterion': [['gini'], ['entropy']]}},
              
              6:
              {'name': 'ExtraTreesClassifier',
               'model': ExtraTreesClassifier(),
               'params': {'n_estimators': [np.arange(1, 102, 2)]}},

              7:
              {'name': 'GaussianNB',
               'model': GaussianNB(),
               'params': {'priors': [[None]]}},
              
              8:
              {'name': 'GradientBoostingClassifier',
               'model': GradientBoostingClassifier(),
               'params': {'loss': [['deviance']],
                          'learning_rate': [learning_rate],
                          'n_estimators': [np.arange(50, 305, 5)]}},
              9:
              {'name': 'KNeighborsClassifier',
               'model': KNeighborsClassifier(),
               'params': {'weights': [['uniform'], ['distance']],
                          'algorithm': [['ball_tree'], ['kd_tree'], ['brute']]}},
             
              10:
              {'name': 'LogisticRegression',
               'model': LogisticRegression(),
               'params': {'penalty': [['l1'], ['l2']],
                          'dual': [[False]],
                          'fit_intercept': [[True], [False]]}},
              11:
              {'name': 'LogisticRegressionCV',
               'model': LogisticRegressionCV(),
               'params': {'penalty': [['l2']],
                          'dual': [[False]],
                          'fit_intercept': [[True], [False]]}},
              12:
              {'name': 'RandomForestClassifier',
               'model': RandomForestClassifier(),
               'params': {'n_estimators': [np.arange(50, 305, 5)]}},
             
              13: 
              {'name': 'SGDClassifier',
               'model':  SGDClassifier(),
               'params': {'loss': [['hinge'], ['log'], ['modified_huber'],
                                   ['squared_hinge'],['perceptron']],
                          'penalty': [['l1'], ['l2'], ['elasticnet'], ['none']],
                          'fit_intercept': [[True], [False]]}},
              
             } 

### Function to gridsearch models and hyperparameters using 5-fold cross-validation and then pickle the best model for use later 

In [53]:
def pipe_check(model_dict, df, y_col, member):
    n_samples = len(df)
    X = df.drop(y_col, axis=1)
    y = df[y_col]


    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
    
    model_name = [] 
    best_score = []
    model_file = [] 
    
    for model in model_dict:
        algo = model_dict[model]['model']
        param_grid = ParameterGrid(model_dict[model]['params'])
        tuned_parameters = param_grid

        scores = ['accuracy']

        for score in scores:
            print ('Model: ', model_dict[model]['name'])
            clf = GridSearchCV(algo, tuned_parameters, cv=5,
                           scoring= score, n_jobs=-1)
            clf.fit(X_train, y_train)

            print ("Best score achieved on development set:")
            print()
            print(clf.best_score_)
            print()
            means = clf.cv_results_['mean_test_score']
            stds = clf.cv_results_['std_test_score']
                
            y_true, y_pred = y_test, clf.predict(X_test)
            path = 'member_models/'
            if len(best_score) == 0:
                file = str(member)+'_model_v1.pkl'
                best_score.append(clf.best_score_)
                model_name.append(model_dict[model]['name'])
                model_file.append(file)
                joblib.dump(clf.best_estimator_, (path+file), compress=1)
            else:
                if clf.best_score_ > max(best_score):
                    v = len(best_score)
                    file = str(member)+'_model_v'+str(v)+'.pkl'
                    best_score.append(clf.best_score_)
                    model_name.append(model_dict[model]['name'])
                    model_file.append(file)
                    joblib.dump(clf.best_estimator_, (path+file), compress=1)
                else:
                    pass 
    output_dict = {'bioguide_id': member, 'best_score': best_score[-1],
                   'model_name': model_name[-1], 'model_file': model_file[-1]}
    return output_dict

In [54]:
def model_manager(member_ids):
    bioguide_ids = []
    best_scores = []
    model_names = []
    model_files = []
    for member in member_ids:
        member_id = member 
        member_votes = vote_df_builder(member, bills, bill_ides)
        member_votes = member_votes.drop(['bill_id', 'vote_id', 'bioguide_id',
                                          'party'], axis=1)
        member_votes = onehot_encoder(member_votes)
        member_votes = member_votes[member_votes['vote'] != 0].reset_index(drop=True)
        results = pipe_check(model_dict, member_votes, 'vote', member_id)
        bioguide_ids.append(results['bioguide_id'])
        best_scores.append(results['best_score'])
        model_names.append(results['model_name'])
        model_files.append(results['model_file'])
    output_dict = {'bioguide_id': bioguide_ids, 'best_score': best_scores,
                   'model_name': model_names, 'model_file': model_files}
    output_df = pd.DataFrame(output_dict)
    return output_df

In [ ]:
member_models = model_manager(member_id_list) 

Model:  AdaBoostClassifier


/home/william/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best score achieved on development set:

0.7185430463576159

Model:  XGBClassifier


/home/william/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best score achieved on development set:

0.7367549668874173

Model:  BaggingClassifier


...

Best score achieved on development set:

0.8115405955243203

Model:  RandomForestClassifier
Best score achieved on development set:

0.7738117255409654

Model:  SGDClassifier


/home/william/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Best score achieved on development set:

0.652672461623821

Model:  AdaBoostClassifier
Best score achieved on development set:

0.8532823010902343

Model:  XGBClassifier
Best score achieved on development set:

0.8558339132451868

Model:  BaggingClassifier
Best score achieved on development set:

0.8321735096265368

Model:  BernoulliNB
Best score achieved on development set:

0.8525864068661564

Model:  DecisionTreeClassifier
Best score achieved on development set:

0.8131524008350731

Model:  ExtraTreesClassifier


/home/william/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best score achieved on development set:

0.8264903734632336

Model:  GaussianNB
Best score achieved on development set:

0.7544653212711668

Model:  GradientBoostingClassifier
